## Load necessary packages

In [14]:
# reloads the packages after each run is an package was editted. We use this since we modify our package while working in this notebook.  
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import pandas as pd

First we scan the dataset in a notebook to see where things might be a little ugly.
In this notebook, we want to do the following things:
1. Look for duplicate observations (remove them if necessary)
2. Look for missing data
3. Seperate categories: The `listed_in` column is a string of different genres or categories, but they are now un-analyzable.
4. Convert dates: The dates are probably strings instead of datetime objects. We need to convert them
5. TV-Shows and Movies are combined in this dataset. We want to split the dataset into a movie dataset and TV show dataset. Then we can correctly edit the duration in either minutes or seasons
6. Last of all, all these functionalities need to be combined in a function

#### Load data

In [2]:
df = pd.read_csv('netflix1.csv')
print(df.shape)
df.head()

(8790, 10)


,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90 min,Documentaries
1,s3,TV Show,Ganglands,Julien Leclercq,France,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
2,s6,TV Show,Midnight Mass,Mike Flanagan,United States,9/24/2021,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries"
3,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,Brazil,9/22/2021,2021,TV-PG,91 min,"Children & Family Movies, Comedies"
4,s8,Movie,Sankofa,Haile Gerima,United States,9/24/2021,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies"


#### Check for duplicate rows

In [3]:
# Check for duplicate rows
if df.duplicated().any():
    print("Warning: The data contains duplicate rows.")
    # Remove duplicate rows
    df.drop_duplicates()
    print("Duplicates removed.")
else:
    print("No duplicates found")

No duplicates found


#### Check for missing values 

In [4]:
# Check for missing values
if df.isnull().values.any():
    print("Warning: The data contains missing values.")
else:
    print("No missing values found")

No missing values found


#### Seperate categories in `listed_in`

In [16]:
# find all unique categories in the column listed in
categories = []
listed_in = df.listed_in.str.split(', ') # per entry: splits all values at ', ' and puts each item in a list 

for cats in listed_in: # sadly, I have not found a way to do this without a double for-loop
    for elem in cats:
        if elem not in categories:
            categories.append(elem)

# we make a new column for every category. This column is 1 if the movie or show was listed in that category and 0 otherwise.        
df_categories = df.iloc[:, :-1]
df_categories.loc[:, categories] = 0

for i in listed_in.index: 
    cats = listed_in.iloc[i]
    # for each entry in listed_in we put a 1 in the column of all categories
    df_categories.loc[i, cats] = 1
    
df_categories.iloc[:5, 10:20] # we can check if everything went well for the first 5 entries

,Crime TV Shows,International TV Shows,TV Action & Adventure,TV Dramas,TV Horror,TV Mysteries,Children & Family Movies,Comedies,Dramas,Independent Movies
0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,0
2,0,0,0,1,1,1,0,0,0,0
3,0,0,0,0,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,1,1


In [20]:
# alternative approach which directly adds the columns to the data frame
# find all unique categories in the column listed in
df_alt=df.copy()
categories = []
listed_in = df_alt.listed_in.str.split(', ') # per entry: splits all values at ', ' and puts each item in a list 
for cats in listed_in: # sadly, I have not found a way to do this without a double for-loop
    for elem in cats:
        if elem not in categories:
            categories.append(elem)
all_categories=categories
for cat in all_categories:
    df_alt[cat] = df_alt['listed_in'].str.contains(cat).astype(int)
    

Of course, we want to migrate this to a function in python. We simply copy the code and paste it into a function.

In [17]:
from netflix_cleaning import clean

In [22]:
df = clean('netflix1.csv')
df.head()

No missing values found
No duplicates found


,show_id,type,title,director,country,date_added,release_year,rating,duration,Documentaries,...,Stand-Up Comedy & Talk Shows,Sports Movies,Science & Nature TV,Cult Movies,Stand-Up Comedy,Faith & Spirituality,LGBTQ Movies,TV Sci-Fi & Fantasy,TV Shows,Classic & Cult TV
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90 min,1,...,0,0,0,0,0,0,0,0,0,0
1,s3,TV Show,Ganglands,Julien Leclercq,France,9/24/2021,2021,TV-MA,1 Season,0,...,0,0,0,0,0,0,0,0,0,0
2,s6,TV Show,Midnight Mass,Mike Flanagan,United States,9/24/2021,2021,TV-MA,1 Season,0,...,0,0,0,0,0,0,0,0,0,0
3,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,Brazil,9/22/2021,2021,TV-PG,91 min,0,...,0,0,0,0,0,0,0,0,0,0
4,s8,Movie,Sankofa,Haile Gerima,United States,9/24/2021,1993,TV-MA,125 min,0,...,0,0,0,0,0,0,0,0,0,0


#### Exercise: convert all dates from strings to datetime objects
Also implement this as a function in netflix_cleaning.py

In [23]:
# Do this by yourself

#### Split tv shows and movies
We discuss two methods for converting duration from string to integer

In [24]:
movies = df[df.type == 'Movie']

#method 1: from each entry, remove the last 4 characters that correspond to ' min'
for i in movies.index:
    movies.loc[i, 'duration'] = int(movies.loc[i, 'duration'][:-4]) # type: ignore # this works just fine!

movies.head(1)

,show_id,type,title,director,country,date_added,release_year,rating,duration,Documentaries,...,Stand-Up Comedy & Talk Shows,Sports Movies,Science & Nature TV,Cult Movies,Stand-Up Comedy,Faith & Spirituality,LGBTQ Movies,TV Sci-Fi & Fantasy,TV Shows,Classic & Cult TV
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90,1,...,0,0,0,0,0,0,0,0,0,0


In [25]:
movies = df[df.type == 'Movie']

#method 2: we do the same thing, but use the build in string functions from pandas. This is much faster!
movies.loc[:, 'duration'] = movies.loc[:, 'duration'].str[:-4].astype(int)
movies.head(1)

,show_id,type,title,director,country,date_added,release_year,rating,duration,Documentaries,...,Stand-Up Comedy & Talk Shows,Sports Movies,Science & Nature TV,Cult Movies,Stand-Up Comedy,Faith & Spirituality,LGBTQ Movies,TV Sci-Fi & Fantasy,TV Shows,Classic & Cult TV
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90,1,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# Returning two dataframes
def seperate_movies_tvshows(df: pd.DataFrame) -> tuple:
    """Seperates the dataset into a movies and tvshows dataset.

    Args:
        df (pd.DataFrame): original data frame with both data about movies and tv shows

    Returns:
        tuple: movies and tv shows dataframe
    """
    movies = df[df.type == 'Movie']
    tvshows = df[df.type == 'TV Show']
    
    movies.loc[:, 'duration'] = movies.loc[:, 'duration'].str[:-4].astype(int)
    tvshows.loc[:, 'duration'] = tvshows.loc[:, 'duration'].str[:-7].astype(int)
    
    return (movies, tvshows)
    

In [27]:
from netflix_cleaning import seperate_movies_tvshows # if you want to import a function from another file

movies, tvshows = seperate_movies_tvshows(df)
tvshows.head(1)

,show_id,type,title,director,country,date_added,release_year,rating,duration,Documentaries,...,Stand-Up Comedy & Talk Shows,Sports Movies,Science & Nature TV,Cult Movies,Stand-Up Comedy,Faith & Spirituality,LGBTQ Movies,TV Sci-Fi & Fantasy,TV Shows,Classic & Cult TV
1,s3,TV Show,Ganglands,Julien Leclercq,France,9/24/2021,2021,TV-MA,1,0,...,0,0,0,0,0,0,0,0,0,0


### Exercise
Add an optional argument `seperate` in the `clean` function. If `seperate` is `True`, seperate the dataset in a movies and tvshows dataset. If not, than return the dataset with both movies and tvshows combined. Use default value `False`. Change the docstring accordingly. 

In [ ]:
# Do this by yourself